In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


import pandas as pd


/tmp/ipykernel_13972/2648110306.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# Inicializa o Firebase Admin, se ainda não estiver inicializado
if not firebase_admin._apps:
    cred = credentials.Certificate('caminho/para/sua/chave-privada-firebase.json')
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [12]:
def listar_colecoes_e_documentos():
    colecoes = db.collections()  # Lista todas as coleções
    for colecao in colecoes:
        print(f'Nome da Coleção: {colecao.id}')
        documentos = colecao.stream()
        for documento in documentos:
            print(f' - Documento ID: {documento.id}, Dados: {documento.to_dict()}')

# Chamada da função
listar_colecoes_e_documentos()

Nome da Coleção: CNPJ
 - Documento ID: 03812120000181_false, Dados: {'CNPJ__c': '03.812.120/0001-81', 'Email': 'borracheiro@teste.com'}
 - Documento ID: 03974743000150_false, Dados: {'CNPJ__c': '03.974.743/0001-50', 'Email': 'tester@teste.com'}
 - Documento ID: 04522171000131_false, Dados: {'CNPJ__c': '04.522.171/0001-31', 'Email': 'caroool@carol.com'}
 - Documento ID: 11934656000106_false, Dados: {'CNPJ__c': '11.934.656/0001-06', 'Email': 'teste1@teste.com'}
 - Documento ID: 16046110000124_false, Dados: {'CNPJ__c': '16.046.110/0001-24', 'Email': 'teste92@teste.com'}
 - Documento ID: 30449012000169_false, Dados: {'CNPJ__c': '30.449.012/0001-69', 'Email': 'n@teste.com'}
 - Documento ID: 35118924000171_false, Dados: {'CNPJ__c': '35.118.924/0001-71', 'Email': 'novoos@teste.com'}
 - Documento ID: 35895109000119_false, Dados: {'CNPJ__c': '35.895.109/0001-19', 'Email': 'nooo@teste.com'}
 - Documento ID: 37668182000183_false, Dados: {'CNPJ__c': '37.668.182/0001-83', 'Email': 'teste@jose.com'}

In [8]:
def colecao_para_dataframe(nome_colecao):
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()

    # Criar uma lista para armazenar os dados dos documentos
    documentos_lista = []

    # Iterar sobre cada documento e adicionar ao dicionário, em seguida, adicionar à lista
    for doc in documentos:
        doc_dict = doc.to_dict()
        doc_dict['document_id'] = doc.id  # Incluir o ID do documento, se necessário
        documentos_lista.append(doc_dict)

    # Converter a lista de dicionários em um DataFrame
    if documentos_lista:
        dataframe = pd.DataFrame(documentos_lista)
    else:
        dataframe = pd.DataFrame()  # Retorna um DataFrame vazio se não houver documentos

    return dataframe


In [10]:
# Exemplo de uso
nome_colecao = 'CPF'
df = colecao_para_dataframe(nome_colecao)
display(df)

,CPF__c,Email,document_id
0,001.553.690-40,josecleiton@jose.com,00155369040_false
1,023.972.440-20,h@teste.com,02397244020_true
2,029.661.230-87,teste1@teste.com,02966123087_false
3,032.938.990-48,testwork@jose.jose,03293899048_false
4,046.408.520-96,caroooooo@b.com,04640852096_false
...,...,...,...
78,883.488.070-63,teste98@teste.com,88348807063_true
79,893.469.240-55,keusantos@jose.com,89346924055_true
80,930.360.100-92,g@gmail.com,93036010092_false
81,986.000.590-70,novoteste@teste.com.br,98600059070_false
